# CHATBOTS - Using Natural Language Processing and Tensorflow

In this Jupyter Notebook, We are going to Build a Chatbot that Understands the Context of Sentense and Respond accordingly.
These are the Things that we are going to do in this Project -
1. Transforming the Conversational Intents into Tensorflow model (Neural Network using TFLEARN) using NLP and Save it as Pickle also.
2. Load the Same Pickle and Model to Build the Framework to Process the Responses.
3. At Last, We Show How the Inputs are Processed and Give the Reponses.
-------------------------------------------------------------------------------------------------------

##### TFLEARN  - TFlearn is a modular and transparent deep learning library built on top of Tensorflow. It was designed to provide a higher-level API to TensorFlow in order to facilitate and speed-up experimentations, while remaining fully transparent and compatible with it.  (http://tflearn.org/)
-------------------------------------------------------------------------------------------------------
##### TENSORFLOW - TensorFlow is an end-to-end open source platform for machine learning. It has a comprehensive, flexible ecosystem of tools, libraries and community resources that lets researchers push the state-of-the-art in ML and developers easily build and deploy ML powered applications.


Libraries ------------------------------------------------------------------------------#FreeBirdsCrew

In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')

#Other
import json
import pickle
import warnings
import random
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [4]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
232 documents
80 classes ['about', 'afternoon', 'anxious', 'ask', 'casual', 'creation', 'death', 'default', 'depressed', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'jokes', 'learn-mental-health', 'learn-more', 'location', 'meditation', 'mental-health-fact', 'morning', 'name', 'neutral-response', 'night', 'no-approach', 'no-response', 'not-talking', 'pandora-useful', 'problem', 'repeat', 'sad', 'scared', 'skill', 'sleep', 'something-else', 'stressed', 'stupid', 'suicide', 'thanks', 'understand', 'user-advice', 'user-agree', 'user-meditation', 'worthless', 'wrong

In [5]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [6]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training, dtype=object)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [7]:
print("Building Neural Network for Our Chatbot to be Contextual....")
model = keras.Sequential()
model.add(layers.Input(shape=(len(train_x[0]),)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(len(train_y[0]), activation='softmax'))

Building Neural Network for Our Chatbot to be Contextual....


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
train_x = np.array(train_x)
train_y = np.array(train_y)
print("Training the Model.......")
model.fit(train_x, train_y, epochs=1000, batch_size=8, verbose=1)
print("Saving the Model.......")
model.save('model.keras')


Training the Model.......
Epoch 1/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0342 - loss: 4.3890       
Epoch 2/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - accuracy: 0.0339 - loss: 4.3660
Epoch 3/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.0313 - loss: 4.3490   
Epoch 4/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - accuracy: 0.0452 - loss: 4.3324   
Epoch 5/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - accuracy: 0.0614 - loss: 4.2756   
Epoch 6/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 676us/step - accuracy: 0.0433 - loss: 4.2419   
Epoch 7/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - accuracy: 0.0299 - loss: 4.1849   
Epoch 8/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.0179 - loss: 4.1034   
Epoch 9/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - accuracy: 0.0182 - loss: 4.0511   
Epoch 10/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.0350 - loss: 3.9814   
Epoch 11/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accurac

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - accuracy: 0.9113 - loss: 0.7180
Epoch 68/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.8573 - loss: 0.8685
Epoch 69/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - accuracy: 0.8550 - loss: 0.9051
Epoch 70/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.8700 - loss: 0.7398
Epoch 71/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - accuracy: 0.8973 - loss: 0.7096
Epoch 72/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.9271 - loss: 0.6695
Epoch 73/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - accuracy: 0.9198 - loss: 0.6577
Epoch 74/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.9144 - loss: 0.6353
Epoch 75/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - accuracy: 0.9371 - loss: 0.5768
Epoch 76/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step - accuracy: 0.9102 - loss: 0.6443
Epoch 77/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step - accuracy: 0.8941 - loss: 0.6245
Epoch 78/1000
29/29 ━━━━━━━━━━━━━━━

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.9770 - loss: 0.1174
Epoch 134/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - accuracy: 0.9773 - loss: 0.1256
Epoch 135/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - accuracy: 0.9797 - loss: 0.1103
Epoch 136/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 660us/step - accuracy: 0.9907 - loss: 0.1021
Epoch 137/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - accuracy: 0.9964 - loss: 0.0890
Epoch 138/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.9872 - loss: 0.1000
Epoch 139/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - accuracy: 0.9915 - loss: 0.0994
Epoch 140/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - accuracy: 0.9952 - loss: 0.0893
Epoch 141/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - accuracy: 0.9741 - loss: 0.1096
Epoch 142/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - accuracy: 0.9900 - loss: 0.0929
Epoch 143/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - accuracy: 0.9904 - loss: 0.0806
Epoch 144/1000
29/29 ━━━━

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - accuracy: 0.9996 - loss: 0.0271
Epoch 200/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - accuracy: 0.9960 - loss: 0.0216
Epoch 201/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.9963 - loss: 0.0300
Epoch 202/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - accuracy: 0.9916 - loss: 0.0263
Epoch 203/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - accuracy: 0.9872 - loss: 0.0302
Epoch 204/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step - accuracy: 0.9921 - loss: 0.0255
Epoch 205/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - accuracy: 0.9969 - loss: 0.0314
Epoch 206/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - accuracy: 0.9994 - loss: 0.0337
Epoch 207/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step - accuracy: 0.9880 - loss: 0.0299
Epoch 208/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - accuracy: 0.9927 - loss: 0.0264
Epoch 209/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - accuracy: 0.9956 - loss: 0.0242
Epoch 210/1000
29/29 ━━━━

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.9936 - loss: 0.0183
Epoch 266/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - accuracy: 0.9959 - loss: 0.0129
Epoch 267/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step - accuracy: 0.9877 - loss: 0.0155
Epoch 268/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step - accuracy: 0.9964 - loss: 0.0075
Epoch 269/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.9991 - loss: 0.0136
Epoch 270/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.9936 - loss: 0.0103
Epoch 271/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 662us/step - accuracy: 0.9965 - loss: 0.0076
Epoch 272/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - accuracy: 0.9862 - loss: 0.0164
Epoch 273/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 695us/step - accuracy: 0.9951 - loss: 0.0129
Epoch 274/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 638us/step - accuracy: 0.9956 - loss: 0.0115
Epoch 275/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.9900 - loss: 0.0126
Epoch 276/1000
29/29 ━━━━

...

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9977 - loss: 0.0017   
Epoch 717/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.9997 - loss: 0.0038   
Epoch 718/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - accuracy: 0.9981 - loss: 0.0049   
Epoch 719/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - accuracy: 0.9997 - loss: 0.0023   
Epoch 720/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - accuracy: 0.9958 - loss: 0.0034   
Epoch 721/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - accuracy: 0.9979 - loss: 0.0056   
Epoch 722/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.9996 - loss: 5.3863e-04
Epoch 723/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - accuracy: 0.9992 - loss: 0.0038   
Epoch 724/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 0.9997 - loss: 0.0040   
Epoch 725/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step - accuracy: 0.9951 - loss: 0.0129
Epoch 726/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.9969 - loss: 0.00

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.9921 - loss: 0.0061   
Epoch 781/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.9919 - loss: 0.0060   
Epoch 782/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - accuracy: 0.9951 - loss: 0.0095   
Epoch 783/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.9929 - loss: 0.0169
Epoch 784/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.9951 - loss: 0.0035   
Epoch 785/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - accuracy: 0.9931 - loss: 0.0050   
Epoch 786/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.9919 - loss: 0.0065   
Epoch 787/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - accuracy: 0.9997 - loss: 0.0016   
Epoch 788/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - accuracy: 0.9917 - loss: 0.0062   
Epoch 789/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step - accuracy: 0.9972 - loss: 0.0021   
Epoch 790/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - accuracy: 0.9762 - loss: 0.019

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.9991 - loss: 0.0025   
Epoch 845/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.9827 - loss: 0.0139   
Epoch 846/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - accuracy: 0.9977 - loss: 0.0050   
Epoch 847/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.9963 - loss: 0.0059   
Epoch 848/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.9992 - loss: 0.0061   
Epoch 849/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.9991 - loss: 0.0030   
Epoch 850/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.9991 - loss: 0.0033   
Epoch 851/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - accuracy: 0.9913 - loss: 0.0066   
Epoch 852/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.9911 - loss: 0.0072   
Epoch 853/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step - accuracy: 0.9985 - loss: 0.0011   
Epoch 854/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.9985 - loss: 0.

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.9916 - loss: 0.0064   
Epoch 909/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - accuracy: 0.9906 - loss: 0.0076   
Epoch 910/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - accuracy: 0.9992 - loss: 0.0023   
Epoch 911/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step - accuracy: 0.9981 - loss: 0.0026   
Epoch 912/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - accuracy: 0.9952 - loss: 0.0037   
Epoch 913/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - accuracy: 0.9983 - loss: 0.0073   
Epoch 914/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - accuracy: 0.9899 - loss: 0.0077   
Epoch 915/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.9985 - loss: 0.0017   
Epoch 916/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - accuracy: 0.9937 - loss: 0.0048   
Epoch 917/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - accuracy: 0.9981 - loss: 0.0015   
Epoch 918/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - accuracy: 0.9977 - loss: 0.

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - accuracy: 0.9960 - loss: 0.0030   
Epoch 973/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - accuracy: 0.9981 - loss: 0.0024   
Epoch 974/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - accuracy: 0.9942 - loss: 0.0087   
Epoch 975/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - accuracy: 0.9939 - loss: 0.0045   
Epoch 976/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.9972 - loss: 0.0039   
Epoch 977/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.9972 - loss: 0.0050   
Epoch 978/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.9966 - loss: 0.0025   
Epoch 979/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - accuracy: 0.9977 - loss: 0.0042   
Epoch 980/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.9987 - loss: 0.0056   
Epoch 981/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.9960 - loss: 0.0029   
Epoch 982/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - accuracy: 0.9992 - loss: 0.

In [10]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [11]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model = keras.models.load_model('model.keras')

Loading Pickle.....
Loading the Model......


In [12]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return np.array(bag)

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict(np.array([bow(sentence, words)]))[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- Hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Hi there. What brings you here today?
You- I am feeling sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
I'm here for you. Could you tell me why you're feeling this way?
You- I'm not sure
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Alright no problem. Is there something you want to talk about?
You- Yes I want to speak about my friend
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
How long have you been feeling this way?
You- Since 2 days ago
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Please don't hesitate to talk to me.
You- I am suicidal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Please don't hesitate to talk to me.
You- I am abused
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
How long have you been feeling this way?
You- now
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Tell me more
You- I am angry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
I'm here for you. Could you tell me why you're feeling this way?
You- I think I am depressed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Sometimes when we